In [ ]:
'''
Author: Ngo Van Uc
Date: 23/08/2024
Gmail: ngovanuc.1508@gmail.com
'''

# 00. Introduction

In [1]:
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from ast import literal_eval
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# 01. Load dataset

In [2]:
train_path = 'E:/IAD/Intern/11_aspectsAnalysis/data/Train.csv'
test_path = 'E:/IAD/Intern/11_aspectsAnalysis/data/Test.csv'
dev_path = 'E:/IAD/Intern/11_aspectsAnalysis/data/Dev.csv'

train_data = pd.read_csv(train_path)
train_data.head()

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


# 02. Preprocessing

In [3]:
class_to_id = {"SCREEN": 0, "CAMERA": 1, "FEATURES": 2, "BATTERY": 3, "PERFORMANCE": 4, "STORAGE": 5, "DESIGN": 6, "PRICE": 7, "GENERAL": 8, "SER&ACC": 9} 
label_to_id = {"Positive": 0, "Negative": 1, "Neutral": 2, "None": 3}

In [5]:
def get_label_dict(sentences_label):
    label_dict = {}
    labels = sentences_label.split(';')[:-1]
    if len(labels) == 1 and labels[-1][1:-1] == "OTHERS":
        return None
    else:
        if labels[-1][1:-1] == "OTHERS":
            for l in labels[:-1]:
                class_name, sentiment = l[1:-1].split('#')
                label_dict[class_name] = sentiment
            return label_dict
        else:
            for l in labels:
                class_name, sentiment = l[1:-1].split('#')
                label_dict[class_name] = sentiment
            return label_dict

In [15]:
sentences = train_data['comment'].tolist()
label_comment = train_data['label'].tolist()
others_label = [[0, 0, 0, 1]] * 10
        
train_set = []

for sentence, sentence_label in zip(sentences, label_comment):
    label_dict = get_label_dict(sentence_label)
    labels_ = []
    if label_dict:
        for aspect in list(class_to_id.keys()):
            train_label = [0] * 4
            if aspect in label_dict.keys():
                label_id = label_to_id[label_dict[aspect]]
                train_label[label_id] =  1
                labels_.append(train_label)
            else:
                labels_.append([0, 0, 0, 1])
        train_set.append(
            {'sentence': sentence, 'labels': labels_}
        )
    else:
        train_set.append(
            {'sentence': sentence, 'labels': others_label}
        )
print(train_set[0]['sentence'])
print(train_set[0]['labels'])

Mới mua máy này Tại thegioididong thốt nốt cảm thấy ok bin trâu chụp ảnh đẹp loa nghe to bắt wf khỏe sóng ổn định, giá thành vừa với túi tiền, nhân viên tư vấn nhiệt tình
[[0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]


In [16]:
for i, data in enumerate(train_set):
    sentence, label = data.items()
    input = np.array(label[-1])
    clear_output()
    if input.shape == (10, 4):
        print("pass")
    else:
        print(i)

pass


# 03. Build model

In [17]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer


class AspectSentimentDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence = self.data[index]['sentence']
        labels = self.data[index]['labels']

        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'sentence_text': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.float)
        }

In [18]:
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')
dataset = AspectSentimentDataset(train_set, tokenizer, max_len=128)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [19]:
dataset.__len__()

7786

In [20]:
import torch.nn as nn
from transformers import BertModel


class AspectSentimentModel(nn.Module):
    def __init__(self, n_classes, n_labels_per_class):
        super(AspectSentimentModel, self).__init__()
        self.bert = BertModel.from_pretrained('google-bert/bert-base-multilingual-cased')
        self.lstm = nn.LSTM(768, 256, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(256*2, n_classes * n_labels_per_class)

    def forward(self, input_ids, attention_mask):
        bert_outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        lstm_outputs, _ = self.lstm(bert_outputs.last_hidden_state)
        lstm_outputs = lstm_outputs[:, -1, :]
        output = self.classifier(lstm_outputs)
        return output.view(-1, 10, 4)
    
model = AspectSentimentModel(n_classes=10, n_labels_per_class=4)

# 04. Training model

In [21]:
import torch.optim as optim
import numpy as np


def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model = model.train()
    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = loss_fn(outputs, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return np.mean(losses)


EPOCHS = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
training = True


optimizer = optim.Adam(model.parameters(), lr=2e-5)
loss_fn = nn.BCEWithLogitsLoss()

if training == True:
    for epoch in range(EPOCHS):
        train_loss = train_epoch(model, dataloader, loss_fn, optimizer, device)
        print(f'Epoch {epoch + 1}/{EPOCHS}, Train loss: {train_loss}')

In [23]:
path_save_model = 'E:/IAD/Intern/11_aspectsAnalysis/models'
torch.save(model.state_dict(), path_save_model+'/model_v2.pth')

In [26]:
# model = AspectSentimentModel(n_classes=10, n_labels_per_class=4)
# model.load_state_dict(torch.load(path_save_model+'/model_v2.pth'))
# model.eval()

In [27]:
def predict_sentiment(model, sentence, tokenizer, max_len, device):
    model = model.eval()

    encoding = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    return outputs


def interpret_predictions(predictions):
    aspect_classes = ['SCREEN', 'CAMERA', 'FEATURES', 'BATTERY', 'PERFORMANCE', 'STORAGE', 'DESIGN', 'PRICE', 'GENERAL', 'SER&ACC']
    sentiment_labels = ['Positive', 'Negative', 'Neutral', 'None']

    results = {}

    for i, aspect in enumerate(aspect_classes):
        aspect_sentiments = predictions[0, i].cpu().numpy()
        sentiment_index = aspect_sentiments.argmax()
        results[aspect] = sentiment_labels[sentiment_index]

    return results

# 05. Thực hiện predict trên bộ dữ liệu kiểm thử - test

In [29]:
test_data = pd.read_csv(test_path)
test_data.head()

,index,comment,n_star,date_time,label
0,0,"Điện thoải ổn. Facelock cực nhanh, vân tay ôk ...",5,5/2/2020,{SCREEN#Positive};{FEATURES#Positive};{PERFORM...
1,1,"Mình mới mua vivo91c. Tải ứng dụng ,games nh...",5,14/05/2019,{FEATURES#Negative};{PERFORMANCE#Positive};{SE...
2,2,Xấu đẹp gì ko biết nhưng rất ưng TGdđ phục vụ ...,5,26/03/2020,{DESIGN#Neutral};{SER&ACC#Positive};
3,3,Màn hình hơi lác khi chơi game. Game nặng thì ...,4,4/6/2019,{PERFORMANCE#Negative};{DESIGN#Negative};{OTHE...
4,4,"Nói chung máy đẹp với màn amoled, ổn trong tầm...",4,12/5/2020,{SCREEN#Positive};{BATTERY#Negative};{DESIGN#P...


In [32]:
max_len = 256
idx = 20
sentence_test = test_data.loc[idx, 'comment']
true_label = test_data.loc[idx, 'label']
print(sentence_test)
print(true_label)

predictions = predict_sentiment(model, sentence_test, tokenizer, max_len, device)
results = interpret_predictions(predictions)

print('Aspect Sentiment Prediction:')
for aspect, sentiment in results.items():
    print(f'{aspect}: {sentiment}')

Mọi thứ ok.mình thấy pin trâu chứ có sao đâu.mua đc 2 tuần rồi chỉ có cảm biến vân tay ko nhạy như mong đợi thôi.
{FEATURES#Negative};{BATTERY#Positive};{GENERAL#Positive};
Aspect Sentiment Prediction:
SCREEN: None
CAMERA: None
FEATURES: Negative
BATTERY: Positive
PERFORMANCE: None
STORAGE: None
DESIGN: None
PRICE: None
GENERAL: Positive
SER&ACC: None


In [33]:
predictions_on_test = []

for i in range(len(test_data)):
    print(i)
    sentence_on_test = test_data['comment'][i]
    prediction_on_test = predict_sentiment(model, sentence_on_test, tokenizer, max_len, device)
    results = interpret_predictions(prediction_on_test)
    predictions_on_test.append(results)
    clear_output()

print('All done!')


All done!


In [35]:
true_labels = []

for i in range(len(test_data)):
    outcome = {}
    label = test_data['label'][i]
    label = label.replace('{', '')
    label = label.replace('}', '')
    labels = label.split(';')
    for l in labels:
        try:
            text = l.split('#')
            outcome[text[0]] = text[1]
        except:
            pass
    true_labels.append(outcome)

# 06. Chuẩn hóa nhãn đầu ra để thực hiện đánh giá hiệu suất mô hình

In [39]:
def full_label(true_label, prediction):
    '''true_label: dict
     prediction: dict '''
    for key, value in prediction.items():
        if key not in true_label:
            true_label[key] = 'None'
    return true_label

In [40]:
# mục tiêu là đưa danh sách các nhãn thực tế về dạng đánh giá 10 class như trong dự đoán
true_full_labels = [] 
for idx in range(len(predictions_on_test)):
    true_label = true_labels[idx]
    prediction = predictions_on_test[idx]
    true_full_labels.append(full_label(true_label, prediction))

In [50]:
# data thỉnh thoảng bị sai -..-
# bằng chứng là SER&ACC trong câu này phải là Positive nhưng nhãn là None
# hoặc GENERAL nên là Positive thì nhãn lại là None

print(predictions_on_test[6])
print(true_labels[6])
print(test_data.loc[6, 'comment'])
for key, value in true_labels[6].items():
    print('true: ',key, value)
    print('predicted: ', key, predictions_on_test[6][key])

{'SCREEN': 'None', 'CAMERA': 'None', 'FEATURES': 'None', 'BATTERY': 'None', 'PERFORMANCE': 'None', 'STORAGE': 'None', 'DESIGN': 'None', 'PRICE': 'None', 'GENERAL': 'Positive', 'SER&ACC': 'Positive'}
{'SCREEN': 'None', 'CAMERA': 'None', 'FEATURES': 'None', 'BATTERY': 'None', 'PERFORMANCE': 'None', 'STORAGE': 'None', 'DESIGN': 'None', 'PRICE': 'None', 'GENERAL': 'None', 'SER&ACC': 'None'}
Thật tuyệt máy qua mượt túi thích  như thế giới Di Động có chỗ giá khác túi không hiểu đánh giá 5sao cho thế giới Di Động phương Phú Châu 5sao nhân viên OK
true:  SCREEN None
predicted:  SCREEN None
true:  CAMERA None
predicted:  CAMERA None
true:  FEATURES None
predicted:  FEATURES None
true:  BATTERY None
predicted:  BATTERY None
true:  PERFORMANCE None
predicted:  PERFORMANCE None
true:  STORAGE None
predicted:  STORAGE None
true:  DESIGN None
predicted:  DESIGN None
true:  PRICE None
predicted:  PRICE None
true:  GENERAL None
predicted:  GENERAL Positive
true:  SER&ACC None
predicted:  SER&ACC Posit

# 07. Đánh giá hiệu suất tổng thể mô hình

In [52]:
true_ = []
predicted_ = []

for idx in range(len(predictions_on_test)):
    for key, value in predictions_on_test[idx].items():
        if true_full_labels[idx][key] == 'None' and value == 'None':
            true_.append(0)
            predicted_.append(0)
        elif true_full_labels[idx][key] != 'None' and true_full_labels[idx][key] == value:
            true_.append(1)
            predicted_.append(1)
        elif true_full_labels[idx][key] == 'None' and value != 'None':
            true_.append(0)
            predicted_.append(1)
        elif true_full_labels[idx][key] != 'None' and value == 'None':
            true_.append(1)
            predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9704226066663284
Precision = 0.8779313232830821
Recall = 1.0
f1 = 0.9349983275727506


# 08. Đánh giá hiệu suất nhận diện khía cạnh

### Accuracy, Precision, Recall, F1-score trên toàn bộ khía cạnh

In [54]:
true_ = []
predicted_ = []

for idx in range(len(predictions_on_test)):
    for key, value in predictions_on_test[idx].items():
        if true_full_labels[idx] == 'None' and value == 'None':
            true_.append(0)
            predicted_.append(0)
        elif true_full_labels[idx][key] != 'None' and value != 'None':
            true_.append(1)
            predicted_.append(1)
        elif true_full_labels[idx][key] == 'None' and value != 'None':
            true_.append(0)
            predicted_.append(1)
        elif true_full_labels[idx][key] != 'None' and value == 'None':
            true_.append(1)
            predicted_.append(0)

print(f'Accuracy = {accuracy_score(true_, predicted_)}')
print(f'Precision = {precision_score(true_, predicted_)}')
print(f'Recall = {recall_score(true_, predicted_)}')
print(f'F1-score = {f1_score(true_, predicted_)}')

Precision = 0.8919169447534298
Recall = 0.7157096102350491
F1-score = 0.7941564872895345


### đánh giá hiệu suất khía cạnh trên class SCREEN

In [55]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'SCREEN'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)
        
accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9523381294964028
Precision = 0.8558951965065502
Recall = 0.7286245353159851
f1 = 0.7871485943775101


### đánh giá hiệu suất khía cạnh trên class FEATURES

In [56]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'FEATURES'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)
        
accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9159172661870504
Precision = 0.861878453038674
Recall = 0.8776371308016878
f1 = 0.8696864111498258


### đánh giá hiệu suất khía cạnh trên class PERFORMANCE

In [61]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'PERFORMANCE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)
        
accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.47302158273381295
Precision = 0.0
Recall = 0.0
f1 = 0.0


### đánh giá hiệu suất khía cạnh trên class GENERAL

In [384]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'GENERAL'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)
        
accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.91681654676259
Precision = 1.0
Recall = 0.8818646232439336
f1 = 0.9372242958941296


### đánh giá hiệu suất khía cạnh trên class BATTERY

In [385]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'BATTERY'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)
        
accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9941546762589928
Precision = 1.0
Recall = 0.9878163074039362
f1 = 0.9938708156529938


### đánh giá hiệu suất khía cạnh trên class STORAGE

In [386]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'STORAGE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)
        
accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9878597122302158
Precision = 0.0
Recall = 0.0
f1 = 0.0


### đánh giá hiệu suất khía cạnh trên class DESIGN

In [388]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'DESIGN'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)
        
accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9545863309352518
Precision = 1.0
Recall = 0.7651162790697674
f1 = 0.8669301712779973


### đánh giá hiệu suất khía cạnh trên class PRICE

In [389]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'PRICE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)
        
accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9676258992805755
Precision = 1.0
Recall = 0.8862559241706162
f1 = 0.9396984924623115


### đánh giá hiệu suất khía cạnh trên class SER&ACC

In [390]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'SER&ACC'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)
        
accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9312050359712231
Precision = 1.0
Recall = 0.7695783132530121
f1 = 0.8697872340425532


### đánh giá hiệu suất khía cạnh trên class CAMERA

In [391]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'CAMERA'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)
        
accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.987410071942446
Precision = 1.0
Recall = 0.9557661927330173
f1 = 0.9773828756058158


# 09. Đánh giá hiệu suất nhận diện tình cảm (đa nhãn)

### Accuracy trên toàn bộ data kiểm thử
acc = corrects/(all predictions)

In [374]:
true_ = []
predicted_ = []
accuracy = []

for idx in range(len(true_full_labels)):
    for key, value in true_full_labels[idx].items():
        if value == 'None':
            continue
        elif predictions_on_test[idx][key] != value:
            accuracy.append(0)
        elif predictions_on_test[idx][key] == value:
            accuracy.append(1)
        
print(f'Accuracy = {sum(accuracy)/len(accuracy)}')

Accuracy = 0.6537987679671458


### Precisio, Recall, F1-score tình cảm trên toàn bộ data kiểm thử

In [393]:
from sklearn.metrics import classification_report


y_true = []
y_pred = []

for idx in range(len(true_labels)):
    for key, value in true_labels[idx].items():
        y_true.append(key + "_" + value)
        y_pred.append(key + "_" + predictions_on_test[idx][key])


print(f'Classification report:')
print(classification_report(y_true, y_pred, digits=4))

Classification report:
                      precision    recall  f1-score   support

    BATTERY_negative     0.8523    0.8866    0.8691       397
     BATTERY_neutral     0.0000    0.0000    0.0000        92
        BATTERY_none     0.9889    1.0000    0.9944      1157
    BATTERY_positive     0.8549    0.9481    0.8991       578
     CAMERA_negative     0.8173    0.8173    0.8173       197
      CAMERA_neutral     0.5000    0.0141    0.0274        71
         CAMERA_none     0.9827    1.0000    0.9913      1591
     CAMERA_positive     0.8522    0.9479    0.8975       365
     DESIGN_negative     0.8929    0.2381    0.3759       105
      DESIGN_neutral     0.0000    0.0000    0.0000        28
         DESIGN_none     0.9467    1.0000    0.9726      1794
     DESIGN_positive     0.8738    0.8855    0.8796       297
   FEATURES_negative     0.9051    0.8517    0.8776       526
    FEATURES_neutral     0.0000    0.0000    0.0000        52
       FEATURES_none     0.9420    1.0000    0

In [362]:
precision_micro = precision_score(y_true, y_pred, average='micro')
recall_micro = recall_score(y_true, y_pred, average='micro')
f1_micro = f1_score(y_true, y_pred, average='micro')

precision_macro = precision_score(y_true, y_pred, average='macro')
recall_macro = recall_score(y_true, y_pred, average='macro')
f1_macro = f1_score(y_true, y_pred, average='macro')

precision_weighted = precision_score(y_true, y_pred, average='weighted')
recall_weighted = recall_score(y_true, y_pred, average='weighted')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print("\nOverall Precision, Recall, F1-Score:")
print(f"Micro Precision: {precision_micro:.4f}")
print(f"Micro Recall: {recall_micro:.4f}")
print(f"Micro F1-Score: {f1_micro:.4f}")
print(f"Macro Precision: {precision_macro:.4f}")
print(f"Macro Recall: {recall_macro:.4f}")
print(f"Macro F1-Score: {f1_macro:.4f}")
print(f"Weighted Precision: {precision_weighted:.4f}")
print(f"Weighted Recall: {recall_weighted:.4f}")
print(f"Weighted F1-Score: {f1_weighted:.4f}")


Overall Precision, Recall, F1-Score:
Micro Precision: 0.9550
Micro Recall: 0.9550
Micro F1-Score: 0.9550
Macro Precision: 0.6855
Macro Recall: 0.6003
Macro F1-Score: 0.6332
Weighted Precision: 0.9463
Weighted Recall: 0.9550
Weighted F1-Score: 0.9481


### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh SCREEN

In [361]:
y_true = []
y_pred = []

key = 'SCREEN'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    negative       0.92      0.58      0.71       133
     neutral       0.00      0.00      0.00        17
        none       0.96      1.00      0.98      1922
    positive       0.86      0.82      0.84       152

    accuracy                           0.96      2224
   macro avg       0.69      0.60      0.63      2224
weighted avg       0.95      0.96      0.95      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh FEATURES

In [365]:
y_true = []
y_pred = []

key = 'FEATURES'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    negative       0.91      0.85      0.88       526
     neutral       0.00      0.00      0.00        52
        none       0.94      1.00      0.97      1413
    positive       0.82      0.80      0.81       233

    accuracy                           0.92      2224
   macro avg       0.67      0.66      0.66      2224
weighted avg       0.90      0.92      0.91      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh PERFORMANCE

In [366]:
y_true = []
y_pred = []

key = 'PERFORMANCE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00       454
     neutral       0.00      0.00      0.00       116
        none       0.47      1.00      0.64      1052
    positive       0.00      0.00      0.00       602

    accuracy                           0.47      2224
   macro avg       0.12      0.25      0.16      2224
weighted avg       0.22      0.47      0.30      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh GENERAL

In [367]:
y_true = []
y_pred = []

key = 'GENERAL'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    negative       0.94      0.72      0.82       361
     neutral       0.00      0.00      0.00        83
        none       0.78      1.00      0.88       658
    positive       0.95      0.93      0.94      1122

    accuracy                           0.88      2224
   macro avg       0.67      0.66      0.66      2224
weighted avg       0.86      0.88      0.87      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh CAMERA

In [368]:
y_true = []
y_pred = []

key = 'CAMERA'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    negative       0.82      0.82      0.82       197
     neutral       0.50      0.01      0.03        71
        none       0.98      1.00      0.99      1591
    positive       0.85      0.95      0.90       365

    accuracy                           0.94      2224
   macro avg       0.79      0.69      0.68      2224
weighted avg       0.93      0.94      0.93      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh BATTERY

In [369]:
y_true = []
y_pred = []

key = 'BATTERY'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    negative       0.85      0.89      0.87       397
     neutral       0.00      0.00      0.00        92
        none       0.99      1.00      0.99      1157
    positive       0.85      0.95      0.90       578

    accuracy                           0.92      2224
   macro avg       0.67      0.71      0.69      2224
weighted avg       0.89      0.92      0.91      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh STORAGE

In [370]:
y_true = []
y_pred = []

key = 'STORAGE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         6
     neutral       0.00      0.00      0.00         3
        none       0.99      1.00      0.99      2197
    positive       0.00      0.00      0.00        18

    accuracy                           0.99      2224
   macro avg       0.25      0.25      0.25      2224
weighted avg       0.98      0.99      0.98      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh DESIGN

In [371]:
y_true = []
y_pred = []

key = 'DESIGN'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    negative       0.89      0.24      0.38       105
     neutral       0.00      0.00      0.00        28
        none       0.95      1.00      0.97      1794
    positive       0.87      0.89      0.88       297

    accuracy                           0.94      2224
   macro avg       0.68      0.53      0.56      2224
weighted avg       0.92      0.94      0.92      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh PRICE

In [372]:
y_true = []
y_pred = []

key = 'PRICE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    negative       0.75      0.50      0.60        90
     neutral       0.88      0.75      0.81       361
        none       0.96      1.00      0.98      1591
    positive       0.75      0.81      0.78       182

    accuracy                           0.92      2224
   macro avg       0.84      0.76      0.79      2224
weighted avg       0.92      0.92      0.92      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh SER&ACC

In [373]:
y_true = []
y_pred = []

key = 'SER&ACC'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    negative       0.89      0.48      0.63       201
     neutral       0.00      0.00      0.00        27
        none       0.91      1.00      0.95      1560
    positive       0.94      0.87      0.90       436

    accuracy                           0.92      2224
   macro avg       0.69      0.59      0.62      2224
weighted avg       0.90      0.92      0.90      2224

